In [1]:
import numpy as np
from datetime import datetime


def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


def sigmoid(z):
    
    return 1 / (1+np.exp(-z))

In [2]:
class SimpleClassificationTest:
    
    # constructor
    def __init__(self, xdata, tdata, learning_rate, iteration_count):
            
        # 가중치 W 형상을 자동으로 구하기 위해 입력데이터가 vector 인지,
        # 아니면 matrix 인지 체크 후, 
        # self.xdata 는 무조건 matrix 로 만들어 주면 코드 일관성이 있음
        
        if xdata.ndim == 1:    # vector
            self.xdata = xdata.reshape(len(xdata), 1)
            self.tdata = xdata.reshape(len(tdata), 1)
            
        elif xdata.ndim == 2:  # matrix
            self.xdata = xdata
            self.tdata = tdata
        
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W = np.random.rand(self.xdata.shape[1], 1) 
        self.b = np.random.rand(1)
        
        print("SimpleClassificationTest Object is created")
        
        
    # obtain current W and current b
    def getW_b(self):
        
        return self.W, self.b
    
    
    # loss function
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) ) 
        
    
    # display current error value
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) ) 
    
    
    # predict method
    # 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
    # 입력변수 x : numpy type
    def predict(self, test_data):
    
        z = np.dot(test_data, self.W) + self.b
        y = sigmoid(z)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    
    # train method
    def train(self):
    
        f = lambda x : self.loss_func()

        print("Initial error value = ", self.error_val() )

        start_time = datetime.now()
        
        for step in  range(self.iteration_count):  
    
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % 2000 == 0):
                print("step = ", step, "error value = ", self.error_val() )
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [3]:
# 입력데이터 / 정답데이터 세팅

x_data = np.array( [ [2, 4], [4, 11], [6, 6], [8, 5], [10, 7], [12, 16], [14, 8], [16, 3], [18, 7] ] )
t_data = np.array( [0, 0, 0, 0, 1, 1, 1, 1, 1] ).reshape(9, 1)

print("x_data.shape = ", x_data.shape, ", t_data.shape = ", t_data.shape)

x_data.shape =  (9, 2) , t_data.shape =  (9, 1)


### learning_rate = 1e-2,  반복횟수 100,000번 수행하는 obj1

In [4]:
obj1 = SimpleClassificationTest(x_data, t_data, 1e-2, 100001)

obj1.train()

SimpleClassificationTest Object is created
Initial error value =  34.82967770445934
step =  0 error value =  23.981894506205695
step =  2000 error value =  0.985355348316244
step =  4000 error value =  0.6843878687449099
step =  6000 error value =  0.5495576482818462
step =  8000 error value =  0.4654402246179459
step =  10000 error value =  0.4054385220615895
step =  12000 error value =  0.35962862715071825
step =  14000 error value =  0.32322120840392393
step =  16000 error value =  0.29349503502834956
step =  18000 error value =  0.26873433780533956
step =  20000 error value =  0.24778200529882008
step =  22000 error value =  0.22982101983912873
step =  24000 error value =  0.2142548429570493
step =  26000 error value =  0.20063646736938504
step =  28000 error value =  0.18862381400216224
step =  30000 error value =  0.1779504458457872
step =  32000 error value =  0.16840567790921387
step =  34000 error value =  0.1598206950928773
step =  36000 error value =  0.15205864057246504
ste

In [5]:
test_data = np.array([3, 17])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.1543764] 0


In [6]:
test_data = np.array([5, 8])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.00071404] 0


In [7]:
test_data = np.array([7, 21])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.99999641] 1


In [8]:
test_data = np.array([12, 0])

(real_val, logical_val) = obj1.predict(test_data)

print(real_val, logical_val)

[0.59991871] 1


In [9]:
(W, b) = obj1.getW_b()

print('W = ', W, ', b = ', b)

W =  [[2.40821968]
 [1.15106883]] , b =  [-28.49350972]
